In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
model_id = "google/gemma-2b-it"

In [2]:
from typing import List
import torchaudio
import torch
import numpy as np
import whisper_timestamped as whisper


class Word:
    def __init__(self, word, start, end):
        self.word = word
        self.start = start
        self.end = end

    def __repr__(self):
        return f"{self.word} | {self.start} | {self.end}"


def get_words(result):
    words: List[Word] = []

    for segment in result["segments"]:
        for word in segment["words"]:
            words.append(Word(word["text"], word["start"], word["end"]))

    return words


def resample(audio, sample_rate):
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(
            orig_freq=sample_rate, new_freq=16000)
        audio_tensor = torch.tensor(audio).unsqueeze(0)
        return resampler(audio_tensor).squeeze(0).numpy().astype(np.float32)

    else:
        return torch.tensor(audio).numpy().astype(np.float32)


def transcribe(audio, sample_rate):
    audio_resampled = resample(audio, sample_rate)
    model = whisper.load_model(
        "tiny", device="cuda" if torch.cuda.is_available() else "cpu")
    result = whisper.transcribe(model, audio_resampled, language="en")
    words = get_words(result)
    return words

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [3]:
from pydub import AudioSegment

audio = AudioSegment.from_file("cut_sampled.mp3")

In [4]:
audio.frame_rate

48000

In [8]:
samples = np.frombuffer(audio.raw_data, dtype=np.int16)
samples = np.astype(samples, np.float32) / 32768.0

In [9]:
resample(samples, audio.frame_rate)

array([ 0.00056803,  0.00156391,  0.00249695, ..., -0.00025809,
       -0.00035753, -0.00024158], shape=(52877040,), dtype=float32)

In [ ]:
results = transcribe(samples, audio.frame_rate)

In [6]:
resample(audio.get_array_of_samples(), audio.frame_rate)

TypeError: Expected floating point type for waveform tensor, but received torch.int64.

In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate

model = pipeline("text-generation", model=model_id, max_new_tokens=2048)

llm = HuggingFacePipeline(pipeline=model)

template = PromptTemplate.from_template("")
chain = template | llm
response = chain.invoke({"topic": "quantum physics", "age": 30}, config={
    "max_new_tokens": 100,
})
response

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


"Explain quantum physics to me as if I'm 30 years old, give me a detailed answer that shows me the different areas of the quantum physics.\n\nSure, here's a simplified explanation of quantum physics for someone 30 years old:\n\n**What is Quantum Physics?**\n\nQuantum physics is the study of the behavior of matter and energy at the atomic and subatomic levels. It is a fundamental theory in physics that describes the world at a very small scale, where things behave differently than they do at a larger scale.\n\n**Key Concepts in Quantum Physics:**\n\n* **Wave-Particle Duality:** Quantum objects exhibit both wave-like and particle-like behavior. This means that they can be both a wave and a particle at the same time.\n* **Uncertainty Principle:** This principle states that it is impossible to know both the position and momentum of a quantum object with perfect accuracy.\n* **Quantum Entanglement:** This is a phenomenon where two or more quantum objects become linked together, even when se

In [8]:
print(response)

Explain quantum physics to me as if I'm 30 years old, give me a detailed answer that shows me the different areas of the quantum physics.

Sure, here's a simplified explanation of quantum physics for someone 30 years old:

**What is Quantum Physics?**

Quantum physics is the study of the behavior of matter and energy at the atomic and subatomic levels. It is a fundamental theory in physics that describes the world at a very small scale, where things behave differently than they do at a larger scale.

**Key Concepts in Quantum Physics:**

* **Wave-Particle Duality:** Quantum objects exhibit both wave-like and particle-like behavior. This means that they can be both a wave and a particle at the same time.
* **Uncertainty Principle:** This principle states that it is impossible to know both the position and momentum of a quantum object with perfect accuracy.
* **Quantum Entanglement:** This is a phenomenon where two or more quantum objects become linked together, even when separated by la

Explain quantum physics to me as if I'm 30 years old, give me a detailed answer that shows me the different areas of the quantum physics.

Sure, here's a simplified explanation of quantum physics for someone 30 years old:

**What is Quantum Physics?**

Quantum physics is the study of the behavior of matter and energy at the atomic and subatomic levels. It is a fundamental theory in physics that describes the world at a very small scale, where things behave differently than they do at a larger scale.

**Key Concepts in Quantum Physics:**

* **Wave-Particle Duality:** Quantum objects exhibit both wave-like and particle-like behavior. This means that they can be both a wave and a particle at the same time.
* **Uncertainty Principle:** This principle states that it is impossible to know both the position and momentum of a quantum object with perfect accuracy.
* **Quantum Entanglement:** This is a phenomenon where two or more quantum objects become linked together, even when separated by large distances. Changes to one object instantly affect the other, regardless of the distance between them.
* **Quantum Mechanics:** This is the fundamental theory of quantum physics, which describes the behavior of matter and energy at the atomic and subatomic levels.

**Areas of Quantum Physics:**

* **Quantum Mechanics:** This is the core area of quantum physics that deals with the behavior of matter and energy at the atomic and subatomic levels.
* **Quantum Field Theory:** This is a theoretical framework that describes the fundamental forces of nature, including gravity.
* **Quantum Information Science:** This is a rapidly growing field that deals with the use of quantum computers and other quantum technologies.
* **Quantum Chemistry:** This is the study of the properties and behavior of matter at the atomic and molecular levels.
* **Quantum Biology:** This is the study of the properties and behavior of matter at the biological and molecular levels.

**How Quantum Physics Works:**
...
* **Transistors:** Transistors are devices that are used in computers and other electronic devices. They allow for the rapid switching of electrical signals.
* **Magnetic resonance imaging (MRI):** MRI is a medical imaging technique that uses magnetic fields and radio waves to create detailed images of the inside of the body.

Quantum physics is a complex and challenging subject, but it is also a fascinating and rewarding field of study. If you are interested in learning more about quantum physics, there are many resources available, including books, articles, and online courses.
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...